In [1]:
%load_ext autoreload
%autoreload 2

### imports and utilities

In [2]:
!pip install nltk scikit-learn matplotlib scipy -q

You should consider upgrading via the 'c:\program files\python37\python.exe -m pip install --upgrade pip' command.


In [3]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [Errno 11001] getaddrinfo failed>


False

In [4]:
from collections import Counter
from utils import *

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from scipy.special import softmax
from scipy.stats import norm
from scipy.stats import entropy as calculate_entropy

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.datasets import fetch_20newsgroups
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In C:\Program Files\Python37\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In C:\Program Files\Python37\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In C:\Program Files\Python37\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


### load dataset

In [5]:
dataset = "newsgroup"

# total number of samples needed
randomize = False

# retrieve dataset
categories = ['rec.autos', 'talk.politics.mideast', 'alt.atheism', 'sci.space']

all_docs = fetch_20newsgroups(subset='train', shuffle=randomize, remove=('headers', 'footers', 'quotes'), categories=categories)
all_docs, old_labels, categories = all_docs.data, all_docs.target, all_docs.target_names

In [6]:
# dataset = "bbc"

# data = pd.read_csv('bbcsport.csv')

# all_docs = data["text"].to_list()
# old_labels = data["topic"].to_list()
# categories = classes = np.unique(data["topic"]).tolist()

### clean dataset

In [7]:
datasize = 40
min_document_length = 160
max_document_length = 256


index = -1
docs, labels, label_indices = [], [], []

sizes = [0]*len(categories)

with tqdm(total=len(categories)*datasize) as pbar:
    while sum(sizes) < len(categories)*datasize:
        index += 1
        label_index = old_labels[index]
            
        if sizes[label_index] == datasize:
            continue
        
        doc = all_docs[index]
        status, doc, word_count = clean_doc(doc, True)
        
        if not status:
            continue
            
        if min_document_length is not None and len(doc) < min_document_length:
            continue
            
        if max_document_length is not None and len(doc) > max_document_length:
            continue
        
        label_indices.append(label_index)
        labels.append(categories[label_index])
        
        docs.append(doc)
        sizes[label_index] += 1
        pbar.update(1)

labels = np.array(labels)
label_indices = np.array(label_indices)

In [8]:
doc_index = 3
print(f"Topic: {labels[doc_index]}\n{'='*50}\n{docs[doc_index][:512]}")

Topic: rec.autos
not to mention my friend s 54 citroen traction avant with the light switch and dimmer integrate in a single stalk off the steer column those dumb french be apparently copying the japanese before the german


In [9]:
print(sizes)
assert min(sizes) == max(sizes) == datasize

[40, 40, 40, 40]


### Split data

In [47]:
x_train, x_test, y_train, y_test = train_test_split(docs, labels, test_size =.3)

In [48]:
print(f"there are {len(docs)} total docs, {len(y_train)} train and {len(y_test)} test")

there are 160 total docs, 112 train and 48 test


### Initialize Vectorizer

In [49]:
# initialize the count vectorizer
vectorizer = CountVectorizer()

# fit it to dataset
vectorizer.fit(x_train)

vocabulary = np.array(vectorizer.get_feature_names())
print("word_count is", len(vocabulary))

word_count is 1463


### Prepare Datatset

In [58]:
# create doc count vectors
train_doc_vectors = vectorizer.transform(x_train).toarray()
test_doc_vectors = vectorizer.transform(x_test).toarray()

wdf_train = pd.DataFrame(train_doc_vectors, columns=vocabulary)
wdf_test = pd.DataFrame(test_doc_vectors, columns=vocabulary)

### some new method

In [59]:
wdf_train.head()

,000,031349,11,12,14,16th,1900,1940,1987,1990,...,yeah,year,yes,yo,yorker,you,your,yourself,zeuge,zuma
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [71]:
n_clusters = 4

wdf_train_prime = wdf_train.copy()
wdf_test_prime = wdf_test.copy()
    
# cluster the topics
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(wdf_train_prime)
print("wdf_train, kmeans_inertia = ", kmeans.inertia_)

clusters = []
categories_prime = list(range(n_clusters))

y_train_prime = kmeans.predict(wdf_train_prime)
y_test_prime = kmeans.predict(wdf_test_prime)

for ii in range(n_clusters):
    clusters.append(Counter(y_train[y_train_prime == ii]))
    print(clusters[-1])

print()

wdf_train, kmeans_inertia =  4158.145675675674
Counter({'alt.atheism': 5, 'talk.politics.mideast': 4, 'rec.autos': 1})
Counter({'alt.atheism': 13, 'talk.politics.mideast': 11, 'sci.space': 7, 'rec.autos': 6})
Counter({'rec.autos': 10, 'sci.space': 7, 'talk.politics.mideast': 5, 'alt.atheism': 3})
Counter({'sci.space': 11, 'alt.atheism': 10, 'rec.autos': 10, 'talk.politics.mideast': 9})



In [72]:
wdf_train_prime.head()

,000,031349,11,12,14,16th,1900,1940,1987,1990,...,yeah,year,yes,yo,yorker,you,your,yourself,zeuge,zuma
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [73]:
# the word topic distr
wdt_train = wdf_train_prime.copy()
wdt_test = wdf_test_prime.copy()

wdt_test["__labels__"] = y_test_prime
wdt_train["__labels__"] = y_train_prime

word_doc_count = wdf_train_prime.sum(0)
topic_doc_count = pd.DataFrame(Counter(y_train_prime), index=[0]).T[0]
word_topic_pr_distr = pd.DataFrame(data=0.0, columns=categories_prime, index=vocabulary)

for category in tqdm(categories_prime):
    pxy = wdf_train_prime[wdt_train["__labels__"] == category].sum(0) / topic_doc_count[category]
    word_topic_pr_distr[category] = pxy * (topic_doc_count[category] / word_doc_count)

print(f"word_topic_pr_distr shape = {word_topic_pr_distr.shape}")


word_topic_pr_distr shape = (1463, 4)


In [82]:
pxy, (topic_doc_count[category] / word_doc_count)

(000         0.025
 031349      0.025
 11          0.000
 12          0.000
 14          0.000
             ...  
 you         0.650
 your        0.175
 yourself    0.000
 zeuge       0.025
 zuma        0.000
 Length: 1463, dtype: float64,
 000         40.000000
 031349      40.000000
 11          20.000000
 12          40.000000
 14          40.000000
               ...    
 you          0.526316
 your         1.739130
 yourself    40.000000
 zeuge       40.000000
 zuma        40.000000
 Length: 1463, dtype: float64)

In [74]:
word_topic_pr_distr.head()

,0,1,2,3
000,0.0,0.0,0.0,1.0
031349,0.0,0.0,0.0,1.0
11,0.0,0.0,1.0,0.0
12,0.0,1.0,0.0,0.0
14,0.0,1.0,0.0,0.0


In [75]:
# the entropy and softma for reducing relation
word_topic_pr_entropy = calculate_entropy(word_topic_pr_distr, axis=1)

# the norm freq of the word in topic
word_topic_pr_freq_weight = word_topic_pr_distr.sum(1)


# word_topic_pr_infl = 1 - softmax(word_topic_pr_entropy)

# influence
# wdf_train_prime *= word_topic_pr_infl

In [76]:
wdf_train_prime.max().max()

7

In [78]:
word_topic_pr_freq_weight.max()

1.0000000000000002

In [57]:
word_topic_pr_entropy

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
wdf_train_prime.head()

In [46]:
# cluster the topics
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(wdf_train_prime)
print("wdf_train, kmeans_inertia = ", kmeans.inertia_)

clusters = []
categories_prime = list(range(n_clusters))

y_train_prime = kmeans.predict(wdf_train_prime)
y_test_prime = kmeans.predict(wdf_test_prime)

for ii in range(n_clusters):
    clusters.append(Counter(y_train[y_train_prime == ii]))
    print(clusters[-1])

print()

wdf_train, kmeans_inertia =  67.20517131884166
Counter({'sci.space': 11, 'rec.autos': 8, 'talk.politics.mideast': 5, 'alt.atheism': 5})
Counter({'alt.atheism': 8, 'talk.politics.mideast': 5, 'rec.autos': 3, 'sci.space': 3})
Counter({'rec.autos': 16, 'alt.atheism': 14, 'talk.politics.mideast': 11, 'sci.space': 10})
Counter({'talk.politics.mideast': 8, 'alt.atheism': 5})



In [41]:
num_iter = 2


print()

for _ in tqdm(range(num_iter)):
    wdt_train_prime = wdf_train_prime.copy()
    wdt_test_prime = wdf_test_prime.copy()

    word_doc_count = wdf_train_prime.sum(0)
    word_word_pr_distr = pd.DataFrame(data=0.0, columns=vocabulary, index=vocabulary)

    for word in tqdm(vocabulary):
        pxy = wdf_train_prime[wdf_train_prime[word] > 0].sum(0) / word_doc_count[word]
        word_word_pr_distr[word] = pxy * (word_doc_count[word] / word_doc_count)

    word_word_pr_distr /= word_word_pr_distr.max().max()
    print(f"word_word_pr_distr shape = {word_word_pr_distr.shape}")

    wdf_train_x = 0 * wdf_train_prime
    wdf_test_x = 0 * wdf_test_prime

    for wx, wo in [(wdf_train_x, wdf_train_prime)]:#, (wdf_test_x, wdf_test_prime)]:
        for doc_index in tqdm(range(len(wo))):
            denom = 0

            indices = (wo.loc[doc_index] > 0)
            xv = wo.loc[doc_index][indices]

            for index, (wordx, word_freq) in enumerate(xv.iteritems()):
                denom += word_freq
                wx.loc[doc_index] += word_freq * word_word_pr_distr.loc[wordx]

            wx.loc[doc_index] /= denom
        wx /= wx.max().max()
    
    wdf_train_prime = wdf_train_x.copy()
    wdf_test_prime = wdf_test_x.copy()
    
    print(wdf_train_prime.head())
    
    
    # the entropy and softma for reducing relation
    word_topic_pr_entropy = calculate_entropy(word_topic_pr_distr, axis=1)
    word_topic_pr_infl = 1 - softmax(word_topic_pr_entropy)
    # word_topic_pr_infl = 1 - (word_topic_pr_entropy / word_topic_pr_entropy.max())
    
    # influence
    wdf_train_prime *= word_topic_pr_infl
    print(wdf_train_prime.head())
    
    # cluster the topics
    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(wdf_train_prime)
    print("wdf_train, kmeans_inertia = ", kmeans.inertia_)

    clusters = []
    categories_prime = list(range(n_clusters))

    y_train_prime = kmeans.predict(wdf_train_prime)
    y_test_prime = kmeans.predict(wdf_test_prime)

    for ii in range(n_clusters):
        clusters.append(Counter(y_train[y_train_prime == ii]))
        print(clusters[-1])

    print()

wdf_train, kmeans_inertia =  67.20517131884166
Counter({'sci.space': 11, 'rec.autos': 8, 'talk.politics.mideast': 5, 'alt.atheism': 5})
Counter({'alt.atheism': 8, 'talk.politics.mideast': 5, 'rec.autos': 3, 'sci.space': 3})
Counter({'rec.autos': 16, 'alt.atheism': 14, 'talk.politics.mideast': 11, 'sci.space': 10})
Counter({'talk.politics.mideast': 8, 'alt.atheism': 5})

     000  031349     11     12  12cyl     14   1900   1940   1968   1984  ...  \
0  0.125   0.125  0.125  0.125  0.125  0.125  0.125  0.125  0.125  0.125  ...   
1  0.125   0.125  0.125  0.125  0.125  0.125  0.125  0.125  0.125  0.125  ...   
2  0.125   0.125  0.125  0.125  0.125  0.125  0.125  0.125  0.125  0.125  ...   
3  0.125   0.125  0.125  0.125  0.125  0.250  0.125  0.125  0.125  0.125  ...   
4  0.125   0.125  0.125  0.125  0.125  0.125  0.125  0.125  0.125  0.125  ...   

    yeah   year    yes     yo  yorker    you   your  yourself  zeuge   zuma  
0  0.125  0.125  0.125  0.125   0.125  0.375  0.125     0.125 


word_word_pr_distr shape = (1440, 1440)



   000  031349   11   12  12cyl   14  1900  1940  1968  1984  ...  yeah  year  \
0  1.0     1.0  1.0  1.0    1.0  1.0   1.0   1.0   1.0   1.0  ...   1.0   1.0   
1  1.0     1.0  1.0  1.0    1.0  1.0   1.0   1.0   1.0   1.0  ...   1.0   1.0   
2  1.0     1.0  1.0  1.0    1.0  1.0   1.0   1.0   1.0   1.0  ...   1.0   1.0   
3  1.0     1.0  1.0  1.0    1.0  1.0   1.0   1.0   1.0   1.0  ...   1.0   1.0   
4  1.0     1.0  1.0  1.0    1.0  1.0   1.0   1.0   1.0   1.0  ...   1.0   1.0   

   yes   yo  yorker  you  your  yourself  zeuge  zuma  
0  1.0  1.0     1.0  1.0   1.0       1.0    1.0   1.0  
1  1.0  1.0     1.0  1.0   1.0       1.0    1.0   1.0  
2  1.0  1.0     1.0  1.0   1.0       1.0    1.0   1.0  
3  1.0  1.0     1.0  1.0   1.0       1.0    1.0   1.0  
4  1.0  1.0     1.0  1.0   1.0       1.0    1.0   1.0  

[5 rows x 1440 columns]



word_topic_pr_distr shape = (1440, 4)
        000    031349        11        12     12cyl        14      1900  \
0  0.999306  0.999306  0.999306  0.999306  0.999306  0.999306  0.999306   
1  0.999306  0.999306  0.999306  0.999306  0.999306  0.999306  0.999306   
2  0.999306  0.999306  0.999306  0.999306  0.999306  0.999306  0.999306   
3  0.999306  0.999306  0.999306  0.999306  0.999306  0.999306  0.999306   
4  0.999306  0.999306  0.999306  0.999306  0.999306  0.999306  0.999306   

       1940      1968      1984  ...      yeah      year       yes        yo  \
0  0.999306  0.999306  0.999306  ...  0.999306  0.999306  0.999306  0.999306   
1  0.999306  0.999306  0.999306  ...  0.999306  0.999306  0.999306  0.999306   
2  0.999306  0.999306  0.999306  ...  0.999306  0.999306  0.999306  0.999306   
3  0.999306  0.999306  0.999306  ...  0.999306  0.999306  0.999306  0.999306   
4  0.999306  0.999306  0.999306  ...  0.999306  0.999306  0.999306  0.999306   

     yorker       you      yo


word_word_pr_distr shape = (1440, 1440)



   000  031349   11   12  12cyl   14  1900  1940  1968  1984  ...  yeah  year  \
0  1.0     1.0  1.0  1.0    1.0  1.0   1.0   1.0   1.0   1.0  ...   1.0   1.0   
1  1.0     1.0  1.0  1.0    1.0  1.0   1.0   1.0   1.0   1.0  ...   1.0   1.0   
2  1.0     1.0  1.0  1.0    1.0  1.0   1.0   1.0   1.0   1.0  ...   1.0   1.0   
3  1.0     1.0  1.0  1.0    1.0  1.0   1.0   1.0   1.0   1.0  ...   1.0   1.0   
4  1.0     1.0  1.0  1.0    1.0  1.0   1.0   1.0   1.0   1.0  ...   1.0   1.0   

   yes   yo  yorker  you  your  yourself  zeuge  zuma  
0  1.0  1.0     1.0  1.0   1.0       1.0    1.0   1.0  
1  1.0  1.0     1.0  1.0   1.0       1.0    1.0   1.0  
2  1.0  1.0     1.0  1.0   1.0       1.0    1.0   1.0  
3  1.0  1.0     1.0  1.0   1.0       1.0    1.0   1.0  
4  1.0  1.0     1.0  1.0   1.0       1.0    1.0   1.0  

[5 rows x 1440 columns]


KeyError: 1

In [39]:
word_topic_pr_infl

array([nan, nan, nan, ..., nan, nan, nan])

In [30]:
np.log1p(1)

0.6931471805599453

In [32]:
np.log(1)

0.0